<a href="https://colab.research.google.com/github/Tommy-Las/WatfordFC/blob/main/Data_cleaning_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages and data

In [ ]:
import pandas as pd
# Turn off warnings
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

In [ ]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import GPS data

In [ ]:
file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/GPS 2018-2023.xlsx'
df_gps = pd.read_excel(file_path)

Import speed data

In [ ]:
file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/max_speed.xlsx'
df_speed = pd.read_excel(file_path)

Import wellbeing data


In [ ]:
#file_path = '/content/drive/MyDrive/WATFORD FC/Datos Wellbeing/wellbeing ssp.xlsx'
#df_wellbeing = pd.read_excel(file_path)

# Merge Sprint values

The sprint values are separated in different rows per session,

we want to merge in into a single row per session.

## Transform data before merging sprint rows

In [ ]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123180 entries, 0 to 123179
Data columns (total 11 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   DATE                                         123180 non-null  object 
 1   ID                                           123180 non-null  int64  
 2   Player Position                              123155 non-null  object 
 3   Max Speed                                    116475 non-null  float64
 4   Sprints                                      113560 non-null  float64
 5   MINUTES                                      113654 non-null  float64
 6   Season                                       123180 non-null  object 
 7   Max Speed Season                             123180 non-null  float64
 8   Avg Speed Season                             122357 non-null  float64
 9   % Max Speed                                  122357 non-nul

We want to remove rows after 08-07-2021 since we don't have data for sprints and speed

In [ ]:
from datetime import date
# Convert DATE column values into Pandas datetime object
df_speed['DATE'] = pd.to_datetime(df_speed['DATE'], dayfirst=True)

# Filter rows after date: '2021-08-07'
date_filter = pd.Timestamp('2021-08-07')
df_speed = df_speed[df_speed['DATE'] > date_filter]

# Filter rows after date: '2023-05-11'
date_filter = pd.Timestamp('2023-05-11')
df_speed = df_speed[df_speed['DATE'] < date_filter]

date_filter = pd.Timestamp('2021-06-28')
df_gps = df_gps[df_gps['DATE'] > date_filter]

In [ ]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55390 entries, 58257 to 113646
Data columns (total 11 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         55390 non-null  datetime64[ns]
 1   ID                                           55390 non-null  int64         
 2   Player Position                              55365 non-null  object        
 3   Max Speed                                    55377 non-null  float64       
 4   Sprints                                      55362 non-null  float64       
 5   MINUTES                                      55389 non-null  float64       
 6   Season                                       55390 non-null  object        
 7   Max Speed Season                             55390 non-null  float64       
 8   Avg Speed Season                             55390 non-null  float64       


Replace NULL sprint values to 0

In [ ]:
# Replace NULL values from Sprint to 0
df_speed['Sprints'] = df_speed['Sprints'].fillna(0)

In [ ]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55390 entries, 58257 to 113646
Data columns (total 11 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         55390 non-null  datetime64[ns]
 1   ID                                           55390 non-null  int64         
 2   Player Position                              55365 non-null  object        
 3   Max Speed                                    55377 non-null  float64       
 4   Sprints                                      55390 non-null  float64       
 5   MINUTES                                      55389 non-null  float64       
 6   Season                                       55390 non-null  object        
 7   Max Speed Season                             55390 non-null  float64       
 8   Avg Speed Season                             55390 non-null  float64       


## Merge sprint values

Print values to verify later

In [ ]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
58257,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
58258,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
58259,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
58260,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
58300,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
58289,2023-05-08,48692,CENTRE BACK,32.56,35.0,69.15,2022-2023,36.21,21.029270,89.919912,54.831811
58290,2023-05-08,87583,CENTRE MIDFIELDER,22.69,11.0,34.13,2022-2023,34.27,21.189939,66.209513,7.079117
58291,2023-05-08,37537,CENTRE MIDFIELDER,28.65,5.0,47.05,2022-2023,34.09,21.974456,84.042241,30.378655
58292,2023-05-08,37537,CENTRE MIDFIELDER,30.59,3.0,22.10,2022-2023,34.09,21.974456,89.733060,39.207088
58293,2023-05-08,37537,CENTRE MIDFIELDER,30.59,8.0,69.15,2022-2023,34.09,21.974456,89.733060,39.207088


Do a group by player and date, and select the maximum value for each session

In [ ]:
df_speed = df_speed.groupby(['DATE', 'ID']).agg('max').reset_index()

In [ ]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
8262,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
8261,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
8260,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
8259,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
8249,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
8240,2023-05-08,10103,FULL BACK,21.24,1.0,44.84,2022-2023,34.95,22.202230,60.772532,-4.333932
8241,2023-05-08,12086,WINGER,35.04,41.0,88.27,2022-2023,35.44,21.901184,98.871332,59.991349
8242,2023-05-08,19817,FULL BACK,31.68,0.0,128.04,2022-2023,35.77,21.605460,88.565837,46.629599
8243,2023-05-08,21079,WINGER,34.51,51.0,88.04,2022-2023,36.97,22.708654,93.345956,51.968499
8244,2023-05-08,23085,CENTRE MIDFIELDER,23.71,7.0,58.52,2022-2023,32.12,20.304388,73.816936,16.772788


# Handle duplicates for GPS Data

In [ ]:
# Define columns to sum
columns_to_sum = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC']

# Define columns to select the first value
columns_to_first = ['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE', 'preseason-season', 'MANAGER']

# Group by the duplicate subset and aggregate
df_gps_aggregated = (
    df_gps[df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)]
    .groupby(['PLAYER', 'DATE'], as_index=False)
    .agg({**{col: 'sum' for col in columns_to_sum},
          **{col: 'first' for col in columns_to_first}})
)

# Ensure non-duplicated rows are preserved by combining them back
df_gps_combined = pd.concat([
    df_gps[~df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)],
    df_gps_aggregated
], ignore_index=True)

# Merge GPS and Speed dataframes

## Prepare the data before the merge

Verify column names for both Dataframes

In [ ]:
df_gps_combined.columns

Index(['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE',
       'preseason-season', 'MANAGER', 'Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC'],
      dtype='object')

In [ ]:
df_speed.columns

Index(['DATE', 'ID', 'Player Position', 'Max Speed', 'Sprints', 'MINUTES',
       'Season', 'Max Speed Season', 'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average'],
      dtype='object')

Change datatypes before the merge

In [ ]:
# Convert DATE column from GPS df into datetime pandas obkect
df_gps_combined['DATE'] = pd.to_datetime(df_gps_combined['DATE'], dayfirst=True)

# Drop NULL values for 'PLAYER'
df_gps_combined = df_gps_combined.dropna(subset=['PLAYER'])

# Convert ID and PLAYER columns to the same data type - integers
df_gps_combined['PLAYER'] = df_gps_combined['PLAYER'].astype(int)
df_speed['ID'] = df_speed['ID'].astype(int)

## Do the merge of both dataframes

In [ ]:
# Perform an inner join on matching DATE and PLAYER/ID values
df_merged = df_gps_combined.merge(df_speed, left_on=['DATE', 'PLAYER'], right_on=['DATE', 'ID'], how='inner')

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7835 non-null   datetime64[ns]
 1   Column2                                      7835 non-null   object        
 2   PLAYER                                       7835 non-null   int64         
 3   Injury                                       7835 non-null   float64       
 4   season                                       7835 non-null   object        
 5   LEAGUE                                       7835 non-null   object        
 6   preseason-season                             7835 non-null   object        
 7   MANAGER                                      7835 non-null   object        
 8   Total D                                      7835 non-null   object        
 9

Verify class imbalances

In [ ]:
print("No Injury: " + str(df_merged[df_merged["Injury"] == 0].shape[0]))
print("Injury: " + str(df_merged[df_merged["Injury"] == 1].shape[0]))
print(f"%: {((df_merged[df_merged['Injury'] == 1].shape[0] / df_merged.shape[0]) * 100):.2f}%")

No Injury: 7774
Injury: 61
%: 0.78%


# Merge df with 'weight' in wellbeing dataframe

In [ ]:
# df_wellbeing = df_wellbeing[["Weight", "Date", "PLAYER"]]
# df_wellbeing.info()

In [ ]:
# # Sort by PLAYER and DATE to ensure correct order for backfilling
# df_wellbeing = df_wellbeing.sort_values(by=['PLAYER', 'Date'])

# # Fill the NULL values in 'Weight' with the previous valid value for each player
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].bfill()
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].ffill()

# df_wellbeing.info()

In [ ]:
# # Merge the DataFrames
# df_merged = df_merged.merge(df_wellbeing, left_on=['DATE', 'PLAYER'], right_on=['Date', 'PLAYER'], how='left')

# # Sort by PLAYER and DATE to ensure chronological order
# df_merged = df_merged.sort_values(by=['PLAYER', 'DATE'])

# # Back-fill null values in 'Weight' for each PLAYER
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].bfill()
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].ffill()

# # Optional: Drop the 'Date' column from the right DataFrame if not needed
# df_merged = df_merged.drop(columns=['Date'])

# # Display the final DataFrame
# df_merged.info()

In [ ]:
#df_merged.groupby('PLAYER')['Weight'].apply(lambda x: x.isnull().sum())

# Transform numeric columns to the same data types

In [ ]:
cols = ['Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC', 'ID', 'Max Speed', 'Sprints', 'MINUTES', 'Max Speed Season',
       'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average']

df_merged[cols] = df_merged[cols].astype(float)

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7835 non-null   datetime64[ns]
 1   Column2                                      7835 non-null   object        
 2   PLAYER                                       7835 non-null   int64         
 3   Injury                                       7835 non-null   float64       
 4   season                                       7835 non-null   object        
 5   LEAGUE                                       7835 non-null   object        
 6   preseason-season                             7835 non-null   object        
 7   MANAGER                                      7835 non-null   object        
 8   Total D                                      7835 non-null   float64       
 9

# Values that are 0 for ACC, DEC, Max Speed

In [ ]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7774
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,3
9,>19.8,420


In [ ]:
# Columns to impute
metrics_imputation_zeroes = ['Total D', 'ACC', 'DEC', 'Max Speed', 'MINUTES']

# Columns to use for KNN calculation
columns_for_knn = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC', 'ID',
                   'Max Speed', 'Sprints', 'MINUTES', '% Max Speed']

# Replace 0 with NaN in the columns to be imputed
df_merged[metrics_imputation_zeroes] = df_merged[metrics_imputation_zeroes].replace(0, np.nan)

# Ensure the columns used for KNN calculation are numeric
columns_for_knn_numeric = df_merged[columns_for_knn].select_dtypes(include=[np.number]).columns.tolist()

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # Adjust n_neighbors if needed

# Perform KNN imputation using the broader set of columns for calculation
imputed_values = knn_imputer.fit_transform(df_merged[columns_for_knn_numeric])

# Update only the specified columns to impute
df_merged[metrics_imputation_zeroes] = imputed_values[:,
    [columns_for_knn_numeric.index(col) for col in metrics_imputation_zeroes]]

In [ ]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7774
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,0
9,>19.8,420


In [ ]:
df_merged[df_merged['% Max Speed'] == 0]

,DATE,Column2,PLAYER,Injury,season,LEAGUE,preseason-season,MANAGER,Total D,>19.8,> 25 Km/h,ACC,DEC,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
2830,2022-10-19,MD (AWAY),77765,1.0,2022-2023,CHAMPIONSHIP,SEASON,BILIC,1646.0,77.0,1.0,13.0,23.0,77765.0,CENTRE MIDFIELDER,20.330,2.0,18.000,2022-2023,32.08,21.291520,0.0,-100.0
3298,2022-09-12,M-1,37990,0.0,2022-2023,CHAMPIONSHIP,SEASON,EDWARDS,5215.0,549.0,50.0,51.0,18.0,37990.0,FULL BACK,32.174,0.0,74.792,2022-2023,37.28,22.669514,0.0,-100.0
5228,2022-04-02,MD,65042,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,9.4,7.6,65042.0,CENTRE MIDFIELDER,23.592,0.0,5.000,2021-2022,34.92,20.048756,0.0,-100.0
5230,2022-04-02,MD,23085,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,3.6,6.2,23085.0,CENTRE MIDFIELDER,17.784,0.0,5.000,2021-2022,30.73,20.278653,0.0,-100.0
5234,2022-04-02,MD,17316,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,6.2,5.6,17316.0,WINGER,21.586,0.0,5.000,2021-2022,33.73,20.441103,0.0,-100.0
5238,2022-04-02,MD,42579,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,23.0,12.6,42579.0,CENTRE BACK,21.274,0.0,5.000,2021-2022,29.76,19.989098,0.0,-100.0
5242,2022-04-02,MD,23081,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,3.6,6.2,23081.0,CENTRE BACK,17.784,0.0,5.000,2021-2022,34.94,20.537766,0.0,-100.0
5244,2022-04-02,MD,37709,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,14.8,6.8,37709.0,CENTRE BACK,21.886,0.0,5.000,2021-2022,36.56,22.374467,0.0,-100.0
6756,2021-11-24,M-5,10452,0.0,2021-2022,PREMIER LEAGUE,SEASON,RANIERI,2354.4,0.0,0.0,18.8,18.0,10452.0,FULL BACK,13.458,0.0,15.000,2021-2022,34.54,22.291234,0.0,-100.0
6887,2021-11-07,MD,21079,0.0,2021-2022,PREMIER LEAGUE,SEASON,RANIERI,10554.0,734.0,361.0,95.0,108.0,21079.0,WINGER,35.094,75.0,100.450,2021-2022,36.54,21.962986,0.0,-100.0


In [ ]:
# Calculate '% Max Speed' for all rows
df_merged['% Max Speed'] = (df_merged['Max Speed'] / df_merged['Max Speed Season']) * 100

# Calculate '% Speed difference against max. Speed average' for all rows
df_merged['%Speed diference against max. Speed average'] = (
    (df_merged['Max Speed'] - df_merged['Avg Speed Season']) / df_merged['Avg Speed Season']
) * 100

In [ ]:
df_merged[(df_merged['PLAYER'] == 65042) & (df_merged['DATE'] == '2022-04-02')]

,DATE,Column2,PLAYER,Injury,season,LEAGUE,preseason-season,MANAGER,Total D,>19.8,> 25 Km/h,ACC,DEC,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
5228,2022-04-02,MD,65042,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,9.4,7.6,65042.0,CENTRE MIDFIELDER,23.592,0.0,5.0,2021-2022,34.92,20.048756,67.560137,17.673139


# Reduce data volume/size

Drop rows from preseason since its not the same intensity as regular season


In [ ]:
# df_merged = df_merged[df_merged['preseason-season'] != 'PRESEASON']

# df_merged.reset_index(drop=True, inplace=True)

Class imbalance

In [ ]:
# print("No Injury: " + str(df_merged[df_merged["Injury"] == 0].shape[0]))
# print("Injury: " + str(df_merged[df_merged["Injury"] == 1].shape[0]))
# print(f"%: {((df_merged[df_merged['Injury'] == 1].shape[0] / df_merged.shape[0]) * 100):.2f}%")

## Players who have never been injured

In [ ]:
# Group by ID and sum the Injury column
injury_counts = df_merged.groupby('ID').agg({'Injury': 'sum'})

# Get the IDs where the sum of Injury is greater than 0
ids_to_remove = injury_counts[injury_counts['Injury'] == 0].index.tolist()

# Remove these IDs from the original DataFrame
#df_filtered = df_merged[~df_merged['ID'].isin(ids_to_remove)]

df_filtered = df_merged.copy()

In [ ]:
print("No Injury: " + str(df_filtered[df_filtered["Injury"] == 0].shape[0]))
print("Injury: " + str(df_filtered[df_filtered["Injury"] == 1].shape[0]))
print(f"%: {((df_filtered[df_filtered['Injury'] == 1].shape[0] / df_filtered.shape[0]) * 100):.2f}%")

No Injury: 7774
Injury: 61
%: 0.78%


Verify at least one player had an injury

In [ ]:
df_filtered.groupby('ID').agg({'Injury': 'sum'})

,Injury
ID,
10103.0,4.0
10452.0,3.0
11111.0,0.0
11721.0,2.0
11833.0,1.0
12086.0,1.0
12345.0,0.0
17316.0,3.0
18096.0,3.0


# Change column names

In [ ]:
column_rename_dict = {
    'Column2': 'Microcycle',
    'DATE': 'DATE',
    'ID': 'PlayerID',
    'Total D': 'TD',
    '>19.8': 'HSR',
    '> 25 Km/h': '+25 Km/h',
    'ACC': 'ACC',
    'DEC': 'DEC',
    'Max Speed': 'Max Speed',
    'Max Speed Season': 'Max Speed Season',
    'Avg Speed Season': 'Avg Speed Season',
    '% Max Speed': '% Max Speed',
    '%Speed diference against max. Speed average': 'Speed Diff Max Avg',
    'Injury': 'Injury',
    'MINUTES': 'Mins',
    'Sprints': 'Sprints',
    'Total D_Rel': 'TD_Rel',
    '>19.8_Rel': 'HSR_Rel',
    '> 25 Km/h_Rel': '+25 Km/h_Rel',
    'ACC_Rel': 'ACC_Rel',
    'DEC_Rel': 'DEC_Rel',
    'Sprints_Rel': 'Sprints_Rel'
}

df_filtered.rename(columns=column_rename_dict, inplace=True)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7835 non-null   datetime64[ns]
 1   Microcycle          7835 non-null   object        
 2   PLAYER              7835 non-null   int64         
 3   Injury              7835 non-null   float64       
 4   season              7835 non-null   object        
 5   LEAGUE              7835 non-null   object        
 6   preseason-season    7835 non-null   object        
 7   MANAGER             7835 non-null   object        
 8   TD                  7835 non-null   float64       
 9   HSR                 7835 non-null   float64       
 10  +25 Km/h            7835 non-null   float64       
 11  ACC                 7835 non-null   float64       
 12  DEC                 7835 non-null   float64       
 13  PlayerID            7835 non-null   float64     

# Drop unnecesary columns

In [ ]:
df_filtered = df_filtered.drop(columns=['season', 'LEAGUE', 'MANAGER', 'PLAYER', 'Player Position', 'Season'])

# Take care of NULL or 0 values

## Max Speed values

In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7835 non-null   datetime64[ns]
 1   Microcycle          7835 non-null   object        
 2   Injury              7835 non-null   float64       
 3   preseason-season    7835 non-null   object        
 4   TD                  7835 non-null   float64       
 5   HSR                 7835 non-null   float64       
 6   +25 Km/h            7835 non-null   float64       
 7   ACC                 7835 non-null   float64       
 8   DEC                 7835 non-null   float64       
 9   PlayerID            7835 non-null   float64       
 10  Max Speed           7835 non-null   float64       
 11  Sprints             7835 non-null   float64       
 12  Mins                7835 non-null   float64       
 13  Max Speed Season    7835 non-null   float64     

In [ ]:
# Specify columns to use for imputation calculation
columns_for_imputation_calculation = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC', 'Sprints', '% Max Speed']

# Specify columns to impute
columns_to_impute = ['Max Speed', 'ACC', 'DEC']

# Initialize the KNNImputer
imputer = KNNImputer(n_neighbors=2, weights="uniform")

# Create a DataFrame with only the relevant columns for imputation calculation
imputation_data = df_filtered[columns_for_imputation_calculation + columns_to_impute]

# Perform the imputation
imputed_data = imputer.fit_transform(imputation_data)

# Update the original DataFrame with the imputed values for the specified columns
df_filtered[columns_to_impute] = imputed_data[:, -len(columns_to_impute):]

# Display DataFrame info to verify changes
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7835 non-null   datetime64[ns]
 1   Microcycle          7835 non-null   object        
 2   Injury              7835 non-null   float64       
 3   preseason-season    7835 non-null   object        
 4   TD                  7835 non-null   float64       
 5   HSR                 7835 non-null   float64       
 6   +25 Km/h            7835 non-null   float64       
 7   ACC                 7835 non-null   float64       
 8   DEC                 7835 non-null   float64       
 9   PlayerID            7835 non-null   float64       
 10  Max Speed           7835 non-null   float64       
 11  Sprints             7835 non-null   float64       
 12  Mins                7835 non-null   float64       
 13  Max Speed Season    7835 non-null   float64     

# Calculate relative values

We want the max and avg values for players that played over 85 in a Match Day

Remove spaces in 'Microcycle' column

In [ ]:
df_filtered['Microcycle'] = df_filtered['Microcycle'].str.replace(' ', '', regex=False)
df_filtered["Microcycle"].unique()

array(['M+2', 'M+1', 'MD', 'M-1', 'M-2', 'M-3', 'M-5', 'M+3', 'M-4',
       'MD(AWAY)', 'MD(HOME)', nan], dtype=object)

In [ ]:
def calculate_relative_values(df_original, metrics):
    # Array that holds match days
    microcycle_values = ['MD', 'MD(HOME)', 'MD(AWAY)']

    # Do a copy of the original DF
    df_copy = df_original.copy()

    # Filter players with +85 mins and match day
    df_filtered_85_md = df_copy[
        (df_copy['Mins'] > 85) &
        (df_copy['Microcycle'].isin(microcycle_values))
    ]

    # Players who never completed 85 mins in matchday
    df_filtered_85_only_mins = df_copy[df_copy['Mins'] > 85]

    # Combine both DataFrames
    df_filtered_85 = pd.concat([df_filtered_85_md, df_filtered_85_only_mins])

    # Temporary dictionaries to hold max and avg values per player
    player_max = {}
    player_avg = {}

    # Calculate max and avg for each player
    for player in df_filtered_85['PlayerID'].unique():
        player_data = df_filtered_85[df_filtered_85['PlayerID'] == player]
        player_max[player] = player_data[metrics].max()
        player_avg[player] = player_data[metrics].mean()

    # Add relative values to the original DataFrame
    for metric in metrics:

        df_original[f"{metric}_Rel"] = np.nan

        for player in df_original['PlayerID'].unique():
            if player in player_max and player in player_avg:
                max_value = player_max[player][metric]
                avg_value = player_avg[player][metric]

                # Compute relative value
                df_original.loc[df_original['PlayerID'] == player, f"{metric}_Rel"] = (
                    (df_original.loc[df_original['PlayerID'] == player, metric] * 100) /
                    ((max_value + avg_value) / 2)
                ).round(2)

    return df_original

Call function to calculate relative values

In [ ]:
metrics_rel = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC']

df_rel = calculate_relative_values(df_filtered, metrics_rel)

In [ ]:
df_rel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7835 non-null   datetime64[ns]
 1   Microcycle          7453 non-null   object        
 2   Injury              7835 non-null   float64       
 3   preseason-season    7835 non-null   object        
 4   TD                  7835 non-null   float64       
 5   HSR                 7835 non-null   float64       
 6   +25 Km/h            7835 non-null   float64       
 7   ACC                 7835 non-null   float64       
 8   DEC                 7835 non-null   float64       
 9   PlayerID            7835 non-null   float64       
 10  Max Speed           7835 non-null   float64       
 11  Sprints             7835 non-null   float64       
 12  Mins                7835 non-null   float64       
 13  Max Speed Season    7835 non-null   float64     

# Function for densities

In [ ]:
cols_mins = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC', 'Sprints']

# Create new columns by dividing by the 'Minutes' column
for col in cols_mins:
    df_rel[f'{col}/Mins'] = df_rel[col] / df_rel['Mins']

In [ ]:
df_rel['Sprints/Mins'] = df_rel['Sprints/Mins'].fillna(0)

# Find Max and Avg values before current date

In [ ]:
def calculate_max_avg(df, metrics):

  # Create copies of the original DataFrame to ensure no unintended changes
  df = df.copy()

  # Loop through each metric to calculate max and avg values
  for metric in metrics:
      # Initialize the max and avg columns for the metric
      max_col = f"{metric}_max"
      avg_col = f"{metric}_avg"
      df[max_col] = None
      df[avg_col] = None

      # Iterate through each row to calculate max and avg based on previous dates
      for idx, row in df.iterrows():
          player_id = row['PlayerID']
          current_date = row['DATE']

          # Filter rows for the same player and only for dates before the current row's date
          player_data_before_date = df[
              (df['PlayerID'] == player_id) &
              (df['DATE'] < current_date)
          ]

          # Compute max and avg for the current metric based on the filtered data
          if not player_data_before_date.empty:
              df.at[idx, max_col] = player_data_before_date[metric].max()
              df.at[idx, avg_col] = player_data_before_date[metric].mean()
          else:
          # Fallback: Use max and avg across all rows for the same player
              player_data = df[df['PlayerID'] == player_id]
              df.at[idx, max_col] = player_data[metric].max()
              df.at[idx, avg_col] = player_data[metric].mean()

  return df


In [ ]:
cols = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC','TD/Mins', 'HSR/Mins', '+25 Km/h/Mins', 'ACC/Mins', 'Sprints/Mins']
#df_max_avg = calculate_max_avg(df_rel, cols)
df_max_avg = df_rel.copy()

In [ ]:
df_max_avg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7835 non-null   datetime64[ns]
 1   Microcycle          7453 non-null   object        
 2   Injury              7835 non-null   float64       
 3   preseason-season    7835 non-null   object        
 4   TD                  7835 non-null   float64       
 5   HSR                 7835 non-null   float64       
 6   +25 Km/h            7835 non-null   float64       
 7   ACC                 7835 non-null   float64       
 8   DEC                 7835 non-null   float64       
 9   PlayerID            7835 non-null   float64       
 10  Max Speed           7835 non-null   float64       
 11  Sprints             7835 non-null   float64       
 12  Mins                7835 non-null   float64       
 13  Max Speed Season    7835 non-null   float64     

# Function for player loads

In [ ]:
#df_max_avg

In [ ]:
def calcular_acumulado(df, columnas_calcular, dias):
    # Create an empty list to store processed player DataFrames
    processed_players = []

    # Process each player separately
    for player_id in df['PlayerID'].unique():
        # Filter data for the current player
        player_data = df[df['PlayerID'] == player_id].copy()

        # Create a full date range for the player (from the first to the last recorded date)
        full_date_range = pd.date_range(start=player_data['DATE'].min(), end=player_data['DATE'].max(), freq='D')

        # Set 'DATE' as the index and reindex to fill missing dates with zeros
        player_data = player_data.set_index('DATE').reindex(full_date_range, fill_value=0).reset_index()
        player_data.rename(columns={'index': 'DATE'}, inplace=True)
        player_data['PlayerID'] = player_id

        # Perform rolling calculations for each metric, excluding the current day
        for dia in dias:
            for col in columnas_calcular:
                # Check if the column exists to avoid errors
                if col in player_data.columns:
                    # Rolling sum, mean, std: exclude the current day using shift(1)
                    player_data[f'{col}-{dia}'] = (
                        player_data[col].shift(1).rolling(window=dia, min_periods=1).sum()
                    )
                    player_data[f'{col}-{dia}-avg'] = (
                        player_data[col].shift(1).rolling(window=dia, min_periods=1).mean()
                    )
                    player_data[f'{col}-{dia}-std'] = (
                        player_data[col].shift(1).rolling(window=dia, min_periods=1).std()
                    )

                    # EWMA: Exclude the current day using shift(1)
                    player_data[f'{col}_EWMA-{dia}'] = (
                        player_data[col].shift(1).ewm(span=dia, adjust=False).mean()
                    )

        # Drop rows where all calculated values are zero (rest days)
        mask_non_zero = (player_data[columnas_calcular].sum(axis=1) > 0)
        player_data = player_data[mask_non_zero]

        # Append the processed player's data to the list
        processed_players.append(player_data)

    # Concatenate all processed player DataFrames into a single DataFrame
    df_resultado = pd.concat(processed_players, ignore_index=True)

    return df_resultado


In [ ]:
cols_calculate = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC', 'Sprints']
#columnas_calcular = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC', 'Sprints'] # Indicamos las columnas que queremos añadir en el df
cumulative_df = calcular_acumulado(df_max_avg, cols_calculate, [7,28]) # loads -1, -3, -7, -21

In [ ]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 70 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7835 non-null   datetime64[ns]
 1   Microcycle          7453 non-null   object        
 2   Injury              7835 non-null   float64       
 3   preseason-season    7835 non-null   object        
 4   TD                  7835 non-null   float64       
 5   HSR                 7835 non-null   float64       
 6   +25 Km/h            7835 non-null   float64       
 7   ACC                 7835 non-null   float64       
 8   DEC                 7835 non-null   float64       
 9   PlayerID            7835 non-null   float64       
 10  Max Speed           7835 non-null   float64       
 11  Sprints             7835 non-null   float64       
 12  Mins                7835 non-null   float64       
 13  Max Speed Season    7835 non-null   float64     

In [ ]:
cumulative_df = cumulative_df.dropna(subset=['TD-7', 'TD-7-std'])
cumulative_df.reset_index(drop=True, inplace=True)

In [ ]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7735 entries, 0 to 7734
Data columns (total 70 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7735 non-null   datetime64[ns]
 1   Microcycle          7357 non-null   object        
 2   Injury              7735 non-null   float64       
 3   preseason-season    7735 non-null   object        
 4   TD                  7735 non-null   float64       
 5   HSR                 7735 non-null   float64       
 6   +25 Km/h            7735 non-null   float64       
 7   ACC                 7735 non-null   float64       
 8   DEC                 7735 non-null   float64       
 9   PlayerID            7735 non-null   float64       
 10  Max Speed           7735 non-null   float64       
 11  Sprints             7735 non-null   float64       
 12  Mins                7735 non-null   float64       
 13  Max Speed Season    7735 non-null   float64     

# Calculate different load metrics

In [ ]:
def calculate_metrics_loads(df, metrics):

  # Calculate ACWR, MSWR for each metric
  for metric in metrics:

      # Calculate 7-day and 28-day averages for ACWR
      df[f'{metric}_ACWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-28-avg']

      # Calculate mean and standard deviation for MSWR
      df[f'{metric}_MSWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-7-std']

  return df

In [ ]:
cols_calculate = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC']
cumulative_df = calculate_metrics_loads(cumulative_df, cols_calculate)

In [ ]:
cumulative_df.drop(columns=['Sprints-28-avg', 'Sprints-28-std', 'DEC-28-avg', 'DEC-28-std', 'ACC-28-avg', 'ACC-28-std','+25 Km/h-28-avg', '+25 Km/h-28-std', 'HSR-28-avg', 'HSR-28-std', 'TD-28-avg', 'TD-28-std',
                            'Sprints-7-avg', 'Sprints-7-std', 'DEC-7-avg', 'DEC-7-std', 'ACC-7-avg', 'ACC-7-std','+25 Km/h-7-avg', '+25 Km/h-7-std', 'HSR-7-avg', 'HSR-7-std', 'TD-7-avg', 'TD-7-std', 'TD-28', 'HSR-28', '+25 Km/h-28', 'ACC-28', 'DEC-28', 'Sprints-28',
                            '+25 Km/h_EWMA-28', 'Sprints_EWMA-28', 'TD_EWMA-28', 'HSR_EWMA-28', 'ACC_EWMA-28', 'DEC_EWMA-28','Sprints_EWMA-7'] , inplace=True)

In [ ]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7735 entries, 0 to 7734
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7735 non-null   datetime64[ns]
 1   Microcycle          7357 non-null   object        
 2   Injury              7735 non-null   float64       
 3   preseason-season    7735 non-null   object        
 4   TD                  7735 non-null   float64       
 5   HSR                 7735 non-null   float64       
 6   +25 Km/h            7735 non-null   float64       
 7   ACC                 7735 non-null   float64       
 8   DEC                 7735 non-null   float64       
 9   PlayerID            7735 non-null   float64       
 10  Max Speed           7735 non-null   float64       
 11  Sprints             7735 non-null   float64       
 12  Mins                7735 non-null   float64       
 13  Max Speed Season    7735 non-null   float64     

In [ ]:
# Exclude 'Microcycle' column for counting NULLs
columns_to_check = cumulative_df.drop(columns=['Microcycle']).columns

# Drop rows where 2 or more null values exist (excluding 'Microcycle')
cumulative_df = cumulative_df[cumulative_df[columns_to_check].notna().sum(axis=1) > (len(columns_to_check) - 1)]

# Display the resulting DataFrame
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7072 entries, 0 to 7734
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7072 non-null   datetime64[ns]
 1   Microcycle          6740 non-null   object        
 2   Injury              7072 non-null   float64       
 3   preseason-season    7072 non-null   object        
 4   TD                  7072 non-null   float64       
 5   HSR                 7072 non-null   float64       
 6   +25 Km/h            7072 non-null   float64       
 7   ACC                 7072 non-null   float64       
 8   DEC                 7072 non-null   float64       
 9   PlayerID            7072 non-null   float64       
 10  Max Speed           7072 non-null   float64       
 11  Sprints             7072 non-null   float64       
 12  Mins                7072 non-null   float64       
 13  Max Speed Season    7072 non-null   float64       
 1

In [ ]:
cumulative_df.describe()

,DATE,Injury,TD,HSR,+25 Km/h,ACC,DEC,PlayerID,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD_Rel,HSR_Rel,+25 Km/h_Rel,ACC_Rel,DEC_Rel,TD-7,TD_EWMA-7,HSR-7,HSR_EWMA-7,+25 Km/h-7,+25 Km/h_EWMA-7,ACC-7,ACC_EWMA-7,DEC-7,DEC_EWMA-7,Sprints-7,TD_ACWR,TD_MSWR,HSR_ACWR,HSR_MSWR,+25 Km/h_ACWR,+25 Km/h_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR
count,7072,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000,7072.000000
mean,2022-07-27 23:46:33.665158400,0.008201,4849.905939,195.270645,39.756363,49.130808,41.371538,44175.912472,27.409549,12.392857,67.095857,34.915169,21.571909,78.570625,27.116018,50.034876,36.358242,22.968867,56.532656,45.990064,21662.907876,3056.436449,868.868708,120.803413,168.110860,22.654512,222.827768,31.788655,183.376284,25.888687,53.708242,1.250076,1.169124,1.260107,0.805877,1.322322,0.588638,1.270539,inf,1.271002,inf
min,2021-08-12 00:00:00,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,10103.000000,1.810000,0.000000,3.780000,28.340000,17.940344,5.890010,-91.074358,0.010000,0.000000,0.000000,0.870000,0.830000,1900.000000,126.487071,10.000000,1.875000,1.000000,0.045287,12.000000,1.372047,1.000000,0.750000,0.000000,0.114668,0.377964,0.022958,0.377964,0.003663,0.377964,0.074364,0.377964,0.053435,0.377964
25%,2022-03-04 00:00:00,0.000000,3149.750000,46.000000,0.000000,31.000000,22.000000,21079.000000,24.847500,1.000000,52.300000,33.880000,20.940727,71.332967,15.520806,32.190000,8.787500,0.000000,36.430000,24.860000,17964.500000,2569.140061,581.000000,82.173109,75.000000,10.706517,175.000000,24.895110,138.000000,19.695520,16.000000,0.911678,0.897492,0.832530,0.624845,0.731011,0.456290,0.905360,0.895013,0.898950,0.798852
50%,2022-08-06 00:00:00,0.000000,4375.500000,128.000000,10.000000,47.000000,37.000000,37990.000000,27.675000,8.000000,67.540000,34.950000,21.716652,79.487625,28.045350,45.310000,23.670000,6.200000,54.490000,41.530000,22146.000000,3083.604524,831.500000,113.878275,142.000000,19.137580,224.000000,31.281841,186.000000,25.682188,55.000000,1.061776,1.055678,1.059777,0.736628,1.103296,0.567081,1.066250,1.057381,1.078408,0.985282
75%,2022-12-27 06:00:00,0.000000,5898.250000,296.250000,57.000000,65.000000,56.000000,66786.000000,30.470000,19.000000,83.780000,36.270000,22.202230,87.536917,41.086216,61.677500,56.790000,34.337500,75.275000,62.727500,25809.500000,3558.180310,1112.000000,151.519880,230.000000,30.617776,269.000000,38.097834,227.450000,31.450412,81.000000,1.333153,1.314039,1.421579,0.876061,1.623332,0.655951,1.374234,1.273041,1.394583,1.190887
max,2023-05-09 00:00:00,1.000000,13837.000000,1610.000000,530.000000,190.000000,171.000000,99471.000000,37.560000,78.000000,141.810000,37.560000,26.496667,100.000000,82.440073,176.810000,347.320000,276.450000,282.140000,321.430000,55983.000000,8428.000000,2918.000000,444.817348,870.000000,133.500000,773.000000,112.065049,653.000000,96.667821,191.000000,4.000000,72.266313,4.000000,97.345034,4.000000,33.234019,4.000000,inf,4.000000,inf
std,NaN,0.090196,2405.301571,193.364045,62.417848,24.590290,25.872552,27583.145382,4.204900,13.979015,22.572436,1.639932,0.863946,11.856174,19.169227,24.637445,35.844334,33.768294,27.470946,28.364558,6814.356972,921.019338,401.925230,55.799658,124.573271,16.505854,79.923607,11.524594,72.088920,10.069182,37.843370,0.664257,1.060601,0.753111,1.418363,0.910076,0.480740,0.687125,NaN,0.697081,NaN


Do KNN imputation for EMWA and ACWR values

In [ ]:
#cumulative_df.columns

In [ ]:
# # Exclude unwanted columns for KNN calculation
# columns_to_exclude = ['DATE', 'Microcycle', 'Injury', 'preseason-season']
# columns_for_knn = [col for col in cumulative_df.columns if col not in columns_to_exclude]

# # Apply KNN Imputation only to selected columns
# imputer = KNNImputer(n_neighbors=3)
# imputed_values = imputer.fit_transform(cumulative_df[columns_for_knn])

# # Replace imputed columns in the original dataframe
# cumulative_df[columns_for_knn] = imputed_values

# cumulative_df.info()

# Do the shift for 7 previous days injury

In [ ]:
def shift_injuries_7_days(df):
  copy_df = df.copy()

    # Initialize a new column for the 7-day injury prediction label
  copy_df['Injury_7_day'] = 0

  # Iterate over the df to assign labels to rows in the 7 days before an injury
  for idx, row in copy_df.iterrows():
      if row['Injury'] == 1:
          # Get the player's ID
          player_id = row['PlayerID']
          injury_date = row['DATE']

          # Assign injury label to the previous 7 days for the same player
          for i in range(1, 8):
              prev_date = pd.to_datetime(injury_date) - pd.Timedelta(days=i)
              mask = (copy_df['PlayerID'] == player_id) & (copy_df['DATE'] == prev_date)
              copy_df.loc[mask, 'Injury_7_day'] = 1

  # Drop the original injury rows
  copy_df = copy_df[copy_df['Injury'] == 0]

  return copy_df

In [ ]:
cumulative_df_inj = shift_injuries_7_days(cumulative_df)

In [ ]:
cumulative_df_inj.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7014 entries, 0 to 7734
Data columns (total 44 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7014 non-null   datetime64[ns]
 1   Microcycle          6682 non-null   object        
 2   Injury              7014 non-null   float64       
 3   preseason-season    7014 non-null   object        
 4   TD                  7014 non-null   float64       
 5   HSR                 7014 non-null   float64       
 6   +25 Km/h            7014 non-null   float64       
 7   ACC                 7014 non-null   float64       
 8   DEC                 7014 non-null   float64       
 9   PlayerID            7014 non-null   float64       
 10  Max Speed           7014 non-null   float64       
 11  Sprints             7014 non-null   float64       
 12  Mins                7014 non-null   float64       
 13  Max Speed Season    7014 non-null   float64       
 1

In [ ]:
print("No Injury: " + str(cumulative_df_inj[cumulative_df_inj["Injury_7_day"] == 0].shape[0]))
print("Injury: " + str(cumulative_df_inj[cumulative_df_inj["Injury_7_day"] == 1].shape[0]))
print(f"%: {((cumulative_df_inj[cumulative_df_inj['Injury_7_day'] == 1].shape[0] / cumulative_df_inj.shape[0]) * 100):.2f}%")

No Injury: 6750
Injury: 264
%: 3.76%


# Remove first 7 days of a player's data

In [ ]:
# Get the minimum date for each player
min_dates = cumulative_df_inj.groupby('PlayerID')['DATE'].transform('min')

# Calculate the cutoff date for each row (min date + 7 days)
cutoff_dates = min_dates + pd.Timedelta(days=7)

# Filter out rows where the date is within the first 7 days for each player
final_df = cumulative_df_inj[cumulative_df_inj['DATE'] >= cutoff_dates]

final_df = cumulative_df_inj.copy()

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7014 entries, 0 to 7734
Data columns (total 44 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7014 non-null   datetime64[ns]
 1   Microcycle          6682 non-null   object        
 2   Injury              7014 non-null   float64       
 3   preseason-season    7014 non-null   object        
 4   TD                  7014 non-null   float64       
 5   HSR                 7014 non-null   float64       
 6   +25 Km/h            7014 non-null   float64       
 7   ACC                 7014 non-null   float64       
 8   DEC                 7014 non-null   float64       
 9   PlayerID            7014 non-null   float64       
 10  Max Speed           7014 non-null   float64       
 11  Sprints             7014 non-null   float64       
 12  Mins                7014 non-null   float64       
 13  Max Speed Season    7014 non-null   float64       
 1

In [ ]:
print("No Injury: " + str(final_df[final_df["Injury_7_day"] == 0].shape[0]))
print("Injury: " + str(final_df[final_df["Injury_7_day"] == 1].shape[0]))
print(f"%: {((final_df[final_df['Injury_7_day'] == 1].shape[0] / final_df.shape[0]) * 100):.2f}%")

No Injury: 6750
Injury: 264
%: 3.76%


# Intensity metric based on Microcycle type

# Remove Preseason data

In [ ]:
final_df = final_df[final_df['preseason-season'] != 'PRESEASON']

final_df.reset_index(drop=True, inplace=True)

In [ ]:
print("No Injury: " + str(final_df[final_df["Injury_7_day"] == 0].shape[0]))
print("Injury: " + str(final_df[final_df["Injury_7_day"] == 1].shape[0]))
print(f"%: {((final_df[final_df['Injury_7_day'] == 1].shape[0] / final_df.shape[0]) * 100):.2f}%")

No Injury: 6297
Injury: 253
%: 3.86%


# Export data

In [ ]:
final_df.to_excel('/content/drive/MyDrive/WATFORD FC/Datos GPS/datos_finales_tommy.xlsx', index=False)

In [ ]:
# result_df.to_csv('/content/drive/MyDrive/WATFORD FC/Datos GPS/datos_finales_tommy.csv', index=False)